# Demostração Selenium

### Passo 1: Instalação da biblioteca

- Abrindo um terminal com python, utilizamos o comando:
  ``` pip install selenium ```

### Passo 2: Importar as bibliotecas:

In [1]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import json
from bs4 import BeautifulSoup
import os

driver = webdriver.Chrome()
# url_paginaInicial = "https://servicositabira.govbr.cloud/pronimtb/"
url_despesas_fonteDeRecurso = "https://servicositabira.govbr.cloud/pronimtb/index.asp?acao=3&item=6"

driver.get(url_despesas_fonteDeRecurso) # Pagina Inicial > Despesas > Fonte de Recurso

### Passo 3: Medidas de segurança para encontrar elementos da página

In [2]:
# Esperar até que o DOM (Document Object Model - a hierarquia HTML da página) esteja carregado
def wait_for_page_load(driver):
  wait = WebDriverWait(driver, 10)
  wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

### Passo 4: Interação com a página - Preenchimento
1. Um input para o usuário selecionar o ano desejado
2. Inspecionar os elementos do menu de seleção e criar variáveis para manipulá-los
3. Compreender o parâmetro 'find_element' e sua estrutura
4. Compreender 'clear' e 'click'
5. Chamada da função

In [3]:
def preencher_pag_inicial(driver):
  global ano_selecionado
  ano_selecionado = input('Qual ano entre 2018 e 2024 deseja raspar? \n\n')
  print('Preenchendo dados da página inicial... \n\n')

  # Dicionário de anos
  anos_dict = {
    '2013': '2013|DW_LC131_FC_9|',
    '2014': '2014|DW_LC131_FC_8|',
    '2015': '2015|DW_LC131_FC_7|',
    '2016': '2016|DW_LC131_FC_6|',
    '2017': '2017|DW_LC131_FC_5|',
    '2018': '2018|DW_LC131_FC_4|',
    '2019': '2019|DW_LC131_FC_3|',
    '2020': '2020|DW_LC131_FC_2|',
    '2021': '2021|DW_LC131_FC_1|',
    '2022': '2022|DW_LC131_FC_0|',
    '2023': '2023|DW_LC131_FC_10|',
    '2024': '2024|DW_LC131_FC_11|'
  }

  if ano_selecionado in anos_dict:
    select_ano = driver.find_element(By.ID, 'cmbAno')
    select = Select(select_ano)
    cmbAno = anos_dict[ano_selecionado]
    select.select_by_value(cmbAno)
  else:
    print(f"Ano inválido: {ano_selecionado}. Por favor, escolha entre 2018 e 2024.")

  # Data inicial
  data_inicial = driver.find_element(By.ID, "txtDataInicial") #Poderia ser By.Text ou ainda By.XPATH
  data_inicial.clear()
  data_inicial.send_keys(f"0101{ano_selecionado}")

  # Data final 
  data_final = driver.find_element(By.ID, "txtDataFinal")
  data_final.clear()
  data_final.send_keys(f"3112{ano_selecionado}")


  # Unidade Gestora
  select_element = driver.find_element(By.ID, 'cmbUnidadeGestora')
  select = Select(select_element)
  select.select_by_value('0')  # Selecionar a opção com valor '0'

  # Clicar no botão "Gerar"
  element = driver.find_element(By.ID, "confirma")
  element.click()
  wait_for_page_load(driver)
  print("Página inicial preenchida! \n \n")

In [4]:
preencher_pag_inicial(driver)

Preenchendo dados da página inicial... 


Página inicial preenchida! 
 



### Passo 5: Localizando informação em uma Tabela Dinâmica

In [5]:
def localizar_cfem_em_fonte_de_recurso(driver):
  print("Localizando CFEM na lista de Fonte de Recursos... \n \n")

  wait_for_page_load(driver)

  # Lista de textos possíveis para o link, adaptando para os anos
  textos_possiveis = [
    "Recursos Ordinários",
    "Recursos Ordinários - Exercício Corrente",
    "Compensação Financeira de Recursos Minerais (CFEM) - Exercício Corrente",
    "Transf União Referente à Compensação Financeira de Recursos Minerais",
    "Transf União CFEM",
    "CFEM"        
    # Adicione outras variações de texto que você espera encontrar
  ]

  link_cfem = None  # Inicializa a variável

  for texto in textos_possiveis:
      try:
          print(f"Tentando texto: {texto}")
          # Pausa até que o usuário pressione Enter
          # input("Pressione Enter para continuar...")

          # Tenta localizar o link pelo texto
          link_cfem = driver.find_element(By.LINK_TEXT, texto)
          break  # Sai do loop se o link for encontrado

      except NoSuchElementException:
          print(f"Link com o texto '{texto}' não encontrado, tentando próximo. \n")
          continue  # Tenta o próximo texto na lista

  if link_cfem:
      link_cfem.click()  # Clica no link encontrado
      wait_for_page_load(driver)
      print("Fonte CFEM encontrada! \n \n \n \n \n")
  else:
    raise Exception("Link CFEM não encontrado. \n \n \n \n \n")

### Passo 6: Localizar e clicar na Fonte CFEM

In [6]:
localizar_cfem_em_fonte_de_recurso(driver)

Localizando CFEM na lista de Fonte de Recursos... 
 

Tentando texto: Recursos Ordinários
Fonte CFEM encontrada! 
 
 
 
 



### 7. Extraindo Tabela Completa para CSV

In [7]:
import pandas as pd

def extrair_tabela_completa_csv(driver, table_xpath, output_file="output.csv"):
    wait_for_page_load(driver)
  
  # Localiza a tabela pelo XPath
    table = driver.find_element(By.XPATH, table_xpath)
  
    # Extrai cabeçalhos
    headers = [header.text for header in table.find_elements(By.XPATH, ".//tr[@class='Class_TituloRel']/th")]

    # Extrai linhas e células
    rows = table.find_elements(By.XPATH, ".//tr[starts-with(@id, 'tbTabela_parLinha')]")
    table_data = []
    for row in rows:
        # Extrai o texto ignorando elementos como links
        cells = row.find_elements(By.XPATH, "./td")
        row_data = [cell.text for cell in cells]
        table_data.append(row_data)

    # Salva os dados no CSV usando pandas
    df = pd.DataFrame(table_data, columns=headers)
    df.to_csv(output_file, index=False, encoding="utf-8")
    print(f"Tabela salva em {output_file}")

In [8]:
extrair_tabela_completa_csv(driver, '//*[@id="tbTabela"]', f"{ano_selecionado}_Recursos_Ordinarios.csv")

Tabela salva em 2018_Recursos_Ordinarios.csv


### 8. Localizando cfem ordinaria

In [8]:
def detalhamento_fonte_de_recurso(driver):
  # Selecionando CFEM na Tabela # Pagina Inicial > Despesas > Fonte de Recurso > CFEM
  wait_for_page_load(driver)
  if ano_selecionado == "2018":
    try:
      # Localize o link pelo texto
      link_cfem = driver.find_element(By.LINK_TEXT, "000.0102 - Rec. Ordinários - CEFEM")
      link_fega = driver.find_element(By.LINK_TEXT, "000.0104 - Rec. Ordinários FEGA")
      link_fundesi = driver.find_element(By.LINK_TEXT, "000.0105 - Rec. Ordinários Fundesi")
      # Aqui existem duas linhas referentes à CFEM > Ao procurar por texto, ele clica no primeiro que aparece
      #cfem_tr_56 = '/html/body/div/div[9]/table[3]/tbody/tr[56]/td[1]/a'
      #cfem_tr_57 = '/html/body/div/div[9]/table[3]/tbody/tr[57]/td[1]/a'

      # no ano de 2023 o xpath muda para
      #cfem_2023_tr_61 = '/html/body/div/div[9]/table[3]/tbody/tr[61]/td[1]/a'
      #cfem_2023_tr_62 = '/html/body/div/div[9]/table[3]/tbody/tr[62]/td[1]/a'

      # Localize o link pelo texto
      #link_cfem = driver.find_element(By.XPATH, cfem_2023_tr_62)

      # Clique no link
      link_cfem.click()

    except TimeoutException:
        print("Ocorreu um timeout ao esperar por um elemento.")
    except NoSuchElementException:
        print("Não foi possível encontrar um elemento.")

In [9]:
detalhamento_fonte_de_recurso(driver)

In [10]:
path_detalhamento_cfem = '//*[@id="tbTabela"]'
extrair_tabela_completa_csv(driver,path_detalhamento_cfem,f"{ano_selecionado}_detalhemento_cfem_ordinario.csv")

Tabela salva em 2018_detalhemento_cfem_ordinario.csv
